# 🔴 **RVC - Disconnected** 🔴

***Modified by [MedicDoesStuff](https://www.youtube.com/@medicdoesstuff/)***

**Notebook hastily written by [Kit Lemonfoot](https://huggingface.co/Kit-Lemonfoot) / [Noel Shirogane's High Flying Birds](https://www.youtube.com/@NoelShiroganesHighFlyingBirds)**

*Based on the work of the [RVC Project](https://github.com/RVC-Project), [Mangio261](https://github.com/Mangio621/), [Kalomaze](https://github.com/kalomaze), [Alexlnkp](https://github.com/alexlnkp), the [Pony Preservation Project](https://boards.4channel.org/mlp/catalog#s=Pony%20Preservation%20Project), and many others in the RVC / voice AI community* ❤

*Notebook version:* **0.24**

---

This notebook is designed to be used with training Retreival-Based Voice Conversion (RVC) models without using a WеbUl. This is done to stay within the scope of Colab's TOS.

⚠️ WARNING: Unlike the original RVC notebook, *this notebook ***does not*** have autosave functionality* due to the massive amount of underlying stress that RVC's autosave features places on Colab to Drive communication. Please be careful and try to keep training sessions short.

# **Dependencies**
🟢 Run this first!

In [ ]:
#@title Install Dependencies
import subprocess

packages = ['build-essential', 'python3-dev', 'ffmpeg', 'aria2']
pip_packages = ['pip', 'setuptools', 'wheel', 'httpx==0.23.0', 'faiss-gpu', 'fairseq', 'ffmpeg', 'ffmpeg-python', 'praat-parselmouth', 'pyworld', 'numpy==1.23.5', 'numba==0.56.4', 'librosa==0.9.2', 'gdown', 'onnxruntime', 'huggingface-hub']
print("Updating and installing system packages...")
for package in packages:
  print(f"Installing {package}...")
  subprocess.check_call(['apt-get', 'install', '-qq', '-y', package])

print("Updating and installing pip packages...")
subprocess.check_call(['pip', 'install', '--upgrade'] + pip_packages)

print('Packages up to date.')
firsttry = True

In [ ]:
#@title Clone Repositories
import os

# READ ME BEFORE CHANGING THINGS
# If you're attempting to replace the imports here with Applio-RVC, it will not work due to requirement discrepancies across the entire notebook.
# I will not be porting this notebook to Applio due to the failure of the Applio team to provide backwards compatibility with the Crepe and Mangio-Crepe f0 feature format.
# DO NOT ASK. IT WILL NOT HAPPEN.

os.chdir('/content/')

if(os.path.exists("/content/Mangio-RVC-Fork")):
  print("RVC already installed, skipping.")
else:
  #Credit to miaaaa0a on the AI Hub Discord for (indirectly) suggesting this variant of Mangio RVC to me.
  !git clone -b pr-optimization --single-branch https://github.com/alexlnkp/Mangio-RVC-Tweaks.git
  #Rename to keep backwards compatibility with old variants of Disconnected
  os.rename("/content/Mangio-RVC-Tweaks", "/content/Mangio-RVC-Fork")
  !git clone https://github.com/maxrmorrison/torchcrepe.git
  !mv torchcrepe/torchcrepe Mangio-RVC-Fork/
  !rm -rf torchcrepe  # Delete the torchcrepe repository folder
  !wget -q https://cdn.discordapp.com/attachments/945486970883285045/1114717554481569802/peppy-generator-388800-07722f17a188.json -O /content/Mangio-RVC-Fork/peppy-generator-388800-07722f17a188.json

os.chdir('/content/Mangio-RVC-Fork')
now_dir = "/content/Mangio-RVC-Fork"
os.makedirs(os.path.join(now_dir, "logs"), exist_ok=True)
os.makedirs(os.path.join(now_dir, "weights"), exist_ok=True)

In [ ]:
#@title Mount Drive
from google.colab import drive
if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')
else:
  print('Drive is already mounted. Proceed.')

os.makedirs('/content/drive/MyDrive/RVC-Disconnected', exist_ok=True)

In [ ]:
#@title Download Pretrained Models
#Didn't ask.

#V1
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0G32k.pth -d /content/Mangio-RVC-Fork/pretrained -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0D32k.pth -d /content/Mangio-RVC-Fork/pretrained -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0G40k.pth -d /content/Mangio-RVC-Fork/pretrained -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0D40k.pth -d /content/Mangio-RVC-Fork/pretrained -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0G48k.pth -d /content/Mangio-RVC-Fork/pretrained -o f0G48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0D48k.pth -d /content/Mangio-RVC-Fork/pretrained -o f0D48k.pth

#V2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0G32k.pth -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0D32k.pth -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0G40k.pth -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0D40k.pth -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0G48k.pth -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0G48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0D48k.pth -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0D48k.pth

#OV2 pretrains
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/poiqazwsx/Ov2Super32kfix/resolve/main/f0Ov2Super32kG.pth -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0G32k_OV2.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/poiqazwsx/Ov2Super32kfix/resolve/main/f0Ov2Super32kD.pth -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0D32k_OV2.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ORVC/Ov2Super/resolve/main/f0Ov2Super40kG.pth?download=true -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0G40k_OV2.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ORVC/Ov2Super/resolve/main/f0Ov2Super40kD.pth?download=true -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0D40k_OV2.pth

#Hubert/RMVPE
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/hubert_base.pt -d /content/Mangio-RVC-Fork -o hubert_base.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/rmvpe.pt -d /content/Mangio-RVC-Fork -o rmvpe.pt

#FM JSONs
!rm -rf /content/Mangio-RVC-Fork/configs/32k.json
!rm -rf /content/Mangio-RVC-Fork/configs/40k.json
!rm -rf /content/Mangio-RVC-Fork/configs/48k.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/32k.json -d /content/Mangio-RVC-Fork/configs -o 32k.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/40k.json -d /content/Mangio-RVC-Fork/configs -o 40k.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/48k.json -d /content/Mangio-RVC-Fork/configs -o 48k.json

In [ ]:
#@title Setup CSVDB
#...Alright, you made your point.
import csv

if not os.path.isdir("csvdb/"):
  os.makedirs("csvdb")
  frmnt, stp = open("csvdb/formanting.csv", "w", newline=""), open("csvdb/stop.csv", "w", newline="")
  csv_writer = csv.writer(frmnt, delimiter=",")
  csv_writer.writerow([False, 1.0, 1.0])
  csv_writer = csv.writer(stp, delimiter=",")
  csv_writer.writerow([False])
  frmnt.close()
  stp.close()

global DoFormant, Quefrency, Timbre
DoFormant, Quefrency, Timbre = False, 1.0, 1.0

# **Set Training Variables**

- `experiment_name`: The name of the model. (i.e: EminemMMLP2)
- `model_architecture`: Specifies the chosen model version. V2 is recommended.
- `target_sample_rate`: Specifies the desired sample rate for the model, The new 40K is recommended for general training.
- `cpu_threads`: Specifies the number of CPU threads to be used during the training. Colab only has 2 so don't even bother changing it.
- `speaker_id`: Represents the ID of the speaker being trained, I recommend not changing it.
- `pitch_extraction_algorithm`: Specifies the algorithm used for pitch extraction from the audio data, rmvpe is recommended.
- `crepe_hop_length`: Represents the hop length parameter used in the Crepe algorithm for pitch extraction, only affects crepe and mangio-crepe.
- `pitch_guidance`: Indicates whether pitch guidance is enabled or disabled for the experiment, can be useful if you want a talking or singing model.
- `use_new_pretrains`: Uses the new Ov2Super pretrained models.

In [ ]:
#@title Set Training Variables
now_dir = "/content/Mangio-RVC-Fork"
experiment_name = "experiment_name" #@param {type:"string"}
path_to_training_folder = "/content/dataset/"
model_architecture = "v2" #@param ["v1","v2"] {allow-input: false}
target_sample_rate = "40k" #@param ["32k", "40k", "48k"] {allow-input: false}
cpu_threads = 2 #@param {type:"integer"}
speaker_id = 0 #@param {type:"integer"}
pitch_extraction_algorithm = "rmvpe" #@param ["harvest", "crepe", "mangio-crepe", "rmvpe"] {allow-input: false}
crepe_hop_length = 64 #@param {type:"integer"}
pitch_guidance = True #@param {type:"boolean"}
use_new_pretrains = True #@param {type:"boolean"}

assert crepe_hop_length!=None, "You need to input something for crepe_hop_length, silly."
assert crepe_hop_length>0, "Hop length must be more than 0."
assert crepe_hop_length<=512, "Save frequency must be less than 512."

if use_new_pretrains:
  assert model_architecture=="v2", "The new pretrains are only for RVC v2. Please change your settings to match or use the old pretrains."
  assert target_sample_rate!="48k", "The new pretrains are only for 32k and 40k at the moment. Please change your settings to match."

if(experiment_name == "experiment_name"):
  print("Warning: Your experiment name should be changed to the name of your dataset.")

#**Preprocessing**
🟢 You should only need to run these cells once per model.

In [ ]:
#@title Load dataset
#@markdown If it doesn't already exist, create a folder in your Google Drive named 'RVC-Disconnected' and place your zip file there. This will look for the following ZIP file inside that 'RVC-Disconnected' folder.
dataset = "zipfile.zip"  #@param {type:"string"}

#@markdown This loader will load datasets in a similar fashion to the SVC dataset loader. For best results, your dataset should be formatted as such:
#@markdown ```
#@markdown zipfile.zip
#@markdown └───character_name
#@markdown     ├───data.wav
#@markdown ```
#@markdown Audio filenames do not matter. All audio files should be in WAV format for best compatibility.

#@markdown You do not need to split the WAV files, preprocessing will automatically do it for you.

import os
import shutil

directories=[]

def sanitize_directory(directory):
  for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    if os.path.isfile(file_path):
      if filename == ".DS_Store" or filename.startswith("._") or not filename.endswith(('.wav', '.flac', '.mp3', '.ogg', '.m4a')):
        os.remove(file_path)
    elif os.path.isdir(file_path):
      #Get rid of the MACOSX directory just so it doesn't mess with renaming later
      if(filename == "__MACOSX"):
        shutil.rmtree(file_path)
        continue
      #Append the directory to directories for future dataset check, then recurse.
      directories.append(file_path)
      sanitize_directory(file_path)

dataset_path = '/content/drive/MyDrive/RVC-Disconnected/' + dataset
final_directory = '/content/dataset'
temp_directory = '/content/temp_dataset'

if os.path.exists(final_directory):
  print("Dataset folder already found. Wiping...")
  shutil.rmtree(final_directory)
if os.path.exists(temp_directory):
  print("Temporary folder already found. Wiping...")
  shutil.rmtree(temp_directory)

if not os.path.exists(dataset_path):
  raise Exception(f'I can\'t find {dataset} in {os.path.dirname(dataset_path)}.')

os.makedirs(final_directory, exist_ok=True)
os.makedirs(temp_directory, exist_ok=True)
#Oops.
!unzip -d "{temp_directory}" -B "{dataset_path}"
print("Sanitizing...")
sanitize_directory(temp_directory)

if(len(directories) == 0):
  #If there's no directories, we're dealing with a ZIP of just audio files.
  #Move everything to /dataset/experiment_name/.
  print("Dataset Type: Audio Files (Single Speaker)")
  expDir=os.path.join(final_directory, experiment_name)
  os.makedirs(expDir, exist_ok=True)
  for r, _, f in os.walk(temp_directory):
    for name in f:
      !cp "{temp_directory}/{name}" "{expDir}"
elif(len(directories) == 1):
  #If there's only one directory, we're dealing with a single speaker.
  #Rename the folder to experiment_name and move it to /dataset/.
  print("Dataset Type: Single Speaker")
  fi = os.path.join(temp_directory, experiment_name)
  os.rename(directories[0], fi)
  shutil.move(fi, final_directory)

else:
  #If anything else, we're dealing with multispeaker.
  #Move all folders to /dataset/ indiscriminately.
  print("Dataset Type: Multispeaker")
  for fi in directories:
    shutil.move(fi, final_directory)

shutil.rmtree(temp_directory)

print("Dataset imported.")


In [ ]:
#@title Preprocessing
# Change the experiment Name and the path to the training folder. You shouldn't need to change anything else.

import os
import subprocess

assert cpu_threads>0, "CPU threads not allocated correctly."

sr = int(target_sample_rate.rstrip('k'))*1000
pttf = path_to_training_folder + experiment_name
os.makedirs("%s/logs/%s" % (now_dir, experiment_name), exist_ok=True)

cmd = "python trainset_preprocess_pipeline_print.py \"%s\" %s %s \"%s/logs/%s\" 1" % (pttf, sr, cpu_threads, now_dir, experiment_name)
print(cmd)
!$cmd

In [ ]:
#@title Feature extraction

pitch_extraction_algorithm = "rmvpe" #@param ["harvest", "crepe", "mangio-crepe", "rmvpe"] {allow-input: false}
#crepe_hop_length = 128 #@param {type:"slider", min:1, max:512, step:1}
#pitch_guidance = True #@param {type:"boolean"}

gpuList = gpus.split("-")
cmd = "python extract_f0_print.py \"%s/logs/%s\" %s %s %s" % (now_dir, experiment_name, cpu_threads, pitch_extraction_algorithm, crepe_hop_length)
print(cmd)
!$cmd

leng = len(gpus)

cmd = "python extract_feature_print.py %s %s %s %s \"%s/logs/%s\" %s" % ("device", leng, 0, 0, now_dir, experiment_name, model_architecture)
print(cmd)
!$cmd


In [ ]:
#@title Save preprocessed dataset files to Google Drive
#Compress dataset folder
loc = "%s/logs/%s" % (now_dir, experiment_name)
!zip -r rvcLogs.zip "{loc}"
DATASET_PATH_DRIVE = "/content/drive/MyDrive/RVC-Disconnected/" + experiment_name
!mkdir -p "{DATASET_PATH_DRIVE}"
!cp /content/Mangio-RVC-Fork/rvcLogs.zip "{DATASET_PATH_DRIVE}"

# **Training**
🟢 Also includes resuming code.

In [ ]:
#@title Load preprocessed dataset files from Google Drive (for resuming)
#@markdown If you already have preprocessed dataset files on Google Drive, you can load them here instead of re-running the preprocessing steps.
import os

BACK_UP_DATASET_PATH = "/content/drive/MyDrive/RVC-Disconnected/" + experiment_name

#Prevent people from loading the ZIP over existing files
ok=True
if(os.path.exists("/content/Mangio-RVC-Fork/logs/"+experiment_name+"/2a_f0")):
  print("Dataset files already loaded, skipping.")
  ok=False

if ok:
  !unzip "{BACK_UP_DATASET_PATH}/rvcLogs.zip" -d /

In [ ]:
#@title Import Model from Drive to Notebook (for resuming)
import os

DATASET_PATH_DRIVE = "/content/drive/MyDrive/RVC-Disconnected/" + experiment_name
DATASET_PATH_COLAB = "/content/Mangio-RVC-Fork/logs/" + experiment_name
#@markdown Input the model's Step Count here (the number located on your model's G and D files.) If you used `save_only_last_ckpt` during training, this number will be 2333333.
STEPCOUNT = 2333333 #@param {type:"integer"}

print("Copying model files...")
!cp "{DATASET_PATH_DRIVE}/D_{STEPCOUNT}.pth" "{DATASET_PATH_COLAB}"
!cp "{DATASET_PATH_DRIVE}/G_{STEPCOUNT}.pth" "{DATASET_PATH_COLAB}"
!cp "{DATASET_PATH_DRIVE}/config.json" "{DATASET_PATH_COLAB}"

print("Copying Tensorboard TFEVENT files...")
for r, _, f in os.walk(DATASET_PATH_DRIVE):
  for name in f:
    if(name.startswith("events.out.tfevents")):
      !cp "{DATASET_PATH_DRIVE}/{name}" "{DATASET_PATH_COLAB}"

print("All done. Welcome back!")

In [ ]:
import os
import math
from random import shuffle

# I will not be adding an autosave feature. Do not ask.

#@title Training
save_frequency = 50 #@param {type:"integer"}
total_epochs = 300 #@param {type:"integer"}
batch_size = 16 #@param {type:"integer"}
save_only_latest_ckpt = True #@param {type:"boolean"}
cache_all_training_sets = False #@param {type:"boolean"}
save_small_final_model = True #@param {type:"boolean"}
#@markdown The automatically calculated log interval is known to be very inaccurate and can cause delays between an epoch finishing and Tensorboard writes. If you would like, you can manually define a log interval here.
use_manual_stepToEpoch = False #@param {type:"boolean"}
manual_stepToEpoch = 000 #@param {type:"integer"}

assert save_frequency!=None, "You need to input something for save_frequency, silly."
assert save_frequency>0, "Save frequency must be more than 0."
if(save_frequency>50):print("...A save frequency of %s? A bit high, but... alright then."%save_frequency)
assert total_epochs!=None, "You need to input something for total_epochs, silly."
assert total_epochs>0, "Total epochs must be more than 0."
if(total_epochs>10000):print("...A total epoch count of of %s? This is going to overtrain, but... alright then."%total_epochs)
assert batch_size!=None, "You need to input something for batch_size, silly."
assert batch_size>0, "Batch size must be more than 0."
assert batch_size<=40, "Batch size must be less than 40. (I'd reccomend a value between 6 and 12 for Colab.)"

pretrained_base = "pretrained/" if model_architecture == "v1" else "pretrained_v2/"
exp_dir = "%s/logs/%s" % (now_dir, experiment_name)
upt = "_OV2" if use_new_pretrains==True else ""

pretrainedD = "%sf0D%s%s.pth" % (pretrained_base, target_sample_rate, upt)
pretrainedG = "%sf0G%s%s.pth" % (pretrained_base, target_sample_rate, upt)

#Log interval
log_interval = 1
liFolderPath = os.path.join(exp_dir, "1_16k_wavs")
if(os.path.exists(liFolderPath) and os.path.isdir(liFolderPath)):
  wav_files = [f for f in os.listdir(liFolderPath) if f.endswith(".wav")]
  if wav_files:
    sample_size = len(wav_files)
    log_interval = math.ceil(sample_size / batch_size)
    if log_interval > 1:
      log_interval += 1

if log_interval > 250 and not use_manual_stepToEpoch:
  print("That's a big dataset you got there. Log interval normalized to 200 steps from %s steps." % log_interval)
  log_interval = 200

if use_manual_stepToEpoch:
  log_interval = manual_stepToEpoch

#Create Python command
cmd = "python train_nsf_sim_cache_sid_load_pretrain.py -e \"%s\" -sr %s -f0 %s -bs %s -g %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s -li %s" % (
    experiment_name,
    target_sample_rate,
    1,
    batch_size,
    0,
    total_epochs,
    save_frequency,
    "-pg %s" % pretrainedG if pretrainedG != "" else "\b",
    "-pd %s" % pretrainedD if pretrainedD != "" else "\b",
    1 if save_only_latest_ckpt else 0,
    1 if cache_all_training_sets else 0,
    1 if save_small_final_model else 0,
    model_architecture,
    log_interval,
)
print(cmd)

#Create mute filelist
exp_dir = "%s/logs/%s" % (now_dir, experiment_name)
gt_wavs_dir = "%s/0_gt_wavs" % (exp_dir)
feature_dir = (
  "%s/3_feature256" % (exp_dir)
  if model_architecture == "v1"
  else "%s/3_feature768" % (exp_dir)
)
f0_dir = "%s/2a_f0" % (exp_dir)
f0nsf_dir = "%s/2b-f0nsf" % (exp_dir)
names = (
  set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
  & set([name.split(".")[0] for name in os.listdir(feature_dir)])
  & set([name.split(".")[0] for name in os.listdir(f0_dir)])
  & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
)
opt = []
for name in names:
  opt.append(
    "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
    % (
      gt_wavs_dir.replace("\\", "\\\\"),
      name,
      feature_dir.replace("\\", "\\\\"),
      name,
      f0_dir.replace("\\", "\\\\"),
      name,
      f0nsf_dir.replace("\\", "\\\\"),
      name,
      speaker_id,
    )
  )
fea_dim = 256 if model_architecture == "v1" else 768
for _ in range(2):
  opt.append(
      "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s/logs/mute/2a_f0/mute.wav.npy|%s/logs/mute/2b-f0nsf/mute.wav.npy|%s"
      % (now_dir, target_sample_rate, now_dir, fea_dim, now_dir, now_dir, speaker_id)
  )
shuffle(opt)
with open("%s/filelist.txt" % exp_dir, "w") as f:
  f.write("\n".join(opt))
print("Mute filelist written. Best of luck training!")


%cd /content/Mangio-RVC-Fork
%load_ext tensorboard
%tensorboard --logdir /content/Mangio-RVC-Fork/logs

os.chdir('/content/Mangio-RVC-Fork')
!$cmd


In [ ]:
#@title Index Training
#@markdown Ensure that Feature Extraction has run successfully before running this cell.

#@markdown **Note: It's rare, but you may encounter a bug with this cell that requires a runtime restart.**
#@markdown **If this happens, restart the runtime, re-run the "Set Training Variables" cell, then re-run this cell.**

#@markdown Use this option if you wish to save the two extra files generated by index training to your Google Drive. (Only the added index is normally needed.)
save_extra_files_to_drive = False #@param {type:"boolean"}

#Oh dear lord why is this baked into infer-web I hate this
import os
import sys
import traceback
import numpy as np
import faiss

#from sklearn.cluster import MiniBatchKMeans

exp_dir = "%s/logs/%s" % (now_dir, experiment_name)
os.makedirs(exp_dir, exist_ok=True)
feature_dir = (
    "%s/3_feature256" % (exp_dir)
    if model_architecture == "v1"
    else "%s/3_feature768" % (exp_dir)
)
print(feature_dir)
if not os.path.exists(feature_dir):
  raise Exception("No features exist for this model yet. Did you run Feature Extraction?")
listdir_res = list(os.listdir(feature_dir))
if len(listdir_res) == 0:
  raise Exception("No features exist for this model yet. Did you run Feature Extraction?")

try:
  from sklearn.cluster import MiniBatchKMeans
except:
  print("Due to a bug with Colab, we will need to reinstall Numpy real quick. Give me a sec!")
  !pip install -U numpy
  print("Numpy reinstalled. Please restart the runtime, and then re-run the \"Set Training Variables\" cell to continue.")
  sys.exit()
else:
  print("Proper Numpy version detected.")

infos=[]
npys=[]
for name in sorted(listdir_res):
  phone = np.load("%s/%s" % (feature_dir, name))
  npys.append(phone)
big_npy = np.concatenate(npys, 0)
big_npy_idx = np.arange(big_npy.shape[0])
np.random.shuffle(big_npy_idx)
if big_npy.shape[0] > 2e5:
  print("Trying doing kmeans %s shape to 10k centers." % big_npy.shape[0])
  try:
    big_npy = (
        MiniBatchKMeans(
            n_clusters=10000,
            verbose=True,
            batch_size=256,
            compute_labels = False,
            init="random"
        )
        .fit(big_npy)
        .cluster_centers_

    )
  except:
    info = traceback.format_exc()
    print(info)

np.save("%s/total_fea.npy" % exp_dir, big_npy)
n_ivf = min(int(16*np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)
print("%s,%s" % (big_npy.shape, n_ivf))
index = faiss.index_factory(256 if model_architecture == "v1" else 768, "IVF%s,Flat" % n_ivf)
print("Training index...")
index_ivf = faiss.extract_index_ivf(index)
index_ivf.nprobe = 1
index.train(big_npy)
faiss.write_index(
    index,
    "%s/trained_IVF%s_Flat_nprobe_%s_%s_%s.index" % (exp_dir, n_ivf, index_ivf.nprobe, experiment_name, model_architecture)
)
print("Adding...")
batch_size_add = 8192
for i in range(0, big_npy.shape[0], batch_size_add):
  index.add(big_npy[i:i+batch_size_add])
faiss.write_index(
    index,
    "%s/added_IVF%s_Flat_nprobe_%s_%s_%s.index"
    % (exp_dir, n_ivf, index_ivf.nprobe, experiment_name, model_architecture)
)

npr = index_ivf.nprobe

print("Saving files to Drive...")
DATASET_PATH_DRIVE = "/content/drive/MyDrive/RVC-Disconnected/" + experiment_name
if(not os.path.exists(DATASET_PATH_DRIVE)):
  !mkdir -p "{DATASET_PATH_DRIVE}"
DATASET_PATH_COLAB = "/content/Mangio-RVC-Fork/logs/" + experiment_name
if(save_extra_files_to_drive):
  !cp "{DATASET_PATH_COLAB}/total_fea.npy" "{DATASET_PATH_DRIVE}"
  !cp "{DATASET_PATH_COLAB}/trained_IVF{n_ivf}_Flat_nprobe_{npr}_{experiment_name}_{model_architecture}.index" "{DATASET_PATH_DRIVE}"
!cp "{DATASET_PATH_COLAB}/added_IVF{n_ivf}_Flat_nprobe_{npr}_{experiment_name}_{model_architecture}.index" "{DATASET_PATH_DRIVE}"

print("All done! Your index file has completed training.")
try:
  firsttry
except:
  print("If you had to restart the runtime, disconnect and delete the runtime in order to continue. (Restarting the runtime again will not work.)")



In [ ]:
#@title Export Model from Notebook to Drive
import os

DATASET_PATH_DRIVE = "/content/drive/MyDrive/RVC-Disconnected/" + experiment_name
DATASET_PATH_COLAB = "/content/Mangio-RVC-Fork/logs/" + experiment_name
if(not os.path.exists(DATASET_PATH_DRIVE)):
  !mkdir -p "{DATASET_PATH_DRIVE}"

#@markdown Use this option if you wish to only copy over weights.
skip_models = False #@param {type:"boolean"}
#@markdown Use these options if you wish to manually input your step count and epoch count for incomplete models. *Do not use this option if your training finished.*
manual_save = False #@param {type:"boolean"}
STEPCOUNT = 000 #@param {type:"integer"}
EPOCHCOUNT = 000 #@param {type:"integer"}

finished=False
potential="/content/Mangio-RVC-Fork/weights/"+experiment_name+".pth"
if os.path.exists(potential):
  finished = True

#VERY hacky. Might break stuff, report to me if it does.
print("Detecting latest model...")
if(not manual_save):
  currentMax = 0
  for r, _, f in os.walk("/content/Mangio-RVC-Fork/weights/"):
    for name in f:
      if(name.endswith(".pth") and (name!=experiment_name+".pth")):
        #Check to see if this PTH is what we're looking for.
        if(name.find(experiment_name)==-1):
          continue
        #Determine Epochcount+Stepcount Phase 1
        pot = name.split('_')
        ep=pot[len(pot)-2][1:]
        #If what we got from the epoch count section of the filename isn't a number, multiple completed models are in weights.
        #Skip it if that happens.
        if(not ep.isdecimal()):
          continue
        #Determine Epochcount+Stepcount Phase 2
        ep=int(ep)
        if ep>currentMax:
          currentMax=ep
          step=pot[len(pot)-1].split('.')
          step=int(step[0][1:])
          EPOCHCOUNT=ep
          STEPCOUNT=step

TSTEP = STEPCOUNT
if(not skip_models):
  print("Copying model files...")
  if(save_only_latest_ckpt):
    TSTEP=2333333
  !cp "{DATASET_PATH_COLAB}/D_{TSTEP}.pth" "{DATASET_PATH_DRIVE}"
  !cp "{DATASET_PATH_COLAB}/G_{TSTEP}.pth" "{DATASET_PATH_DRIVE}"
  !cp "{DATASET_PATH_COLAB}/config.json" "{DATASET_PATH_DRIVE}"

print("Copying Tensorboard TFEVENT files...")
for r, d, f in os.walk(DATASET_PATH_COLAB):
  for name in f:
    if(name.startswith("events.out.tfevents") and os.path.exists(os.path.join(DATASET_PATH_COLAB, name))):
      !cp "{DATASET_PATH_COLAB}/{name}" "{DATASET_PATH_DRIVE}"

print("Copying weight file...")
if(finished):
  !cp "{potential}" "{DATASET_PATH_DRIVE}"
else:
  !cp "/content/Mangio-RVC-Fork/weights/{experiment_name}_e{EPOCHCOUNT}_s{STEPCOUNT}.pth" "{DATASET_PATH_DRIVE}"

print("All done!")

In [ ]:
#@title Export finished Model to HuggingFace
#Credit to LollenApe for this cell!

#TODO:
#maybe remove glob dependency?

import os
import zipfile
import glob

assert experiment_name, "No experiment name found."

#@markdown Use this option if you wish to manually use a specific epoch value. *Leave this blank if your model finished training.*
manual_epoch_number = "" #@param {type:"string"}
num_epochs = int(manual_epoch_number) if manual_epoch_number.isdigit() else None

# Construct the weights path based on the provided number of epochs
if num_epochs is not None:
  weights_path = f"/content/Mangio-RVC-Fork/weights/{experiment_name}_e{num_epochs}*"
else:
  print("Autodetecting epoch count...")
  potential = f"/content/Mangio-RVC-Fork/weights/{experiment_name}.pth"
  if os.path.exists(potential):
    weights_path = f"/content/Mangio-RVC-Fork/weights/{experiment_name}"
  else:
    currentMax = 0
    for r, _, f in os.walk("/content/Mangio-RVC-Fork/weights/"):
      for name in f:
        if(name.endswith(".pth") and (name!=experiment_name+".pth")):
          if(name.find(experiment_name)==-1):
            continue
          pot = name.split('_')
          ep=pot[len(pot)-2][1:]
          if(not ep.isdecimal()):
            continue
          ep=int(ep)
          if ep>currentMax:
            currentMax=ep
    num_epochs=currentMax
    weights_path = f"/content/Mangio-RVC-Fork/weights/{experiment_name}_e{num_epochs}*"
  print(f"Model with path {weights_path} found automatically.")

weights_files = glob.glob(weights_path + ".pth")
logs_path = f"/content/Mangio-RVC-Fork/logs/{experiment_name}/added_*_1_{experiment_name}_{model_architecture}.index"

print(f"Searching for weights files in: {weights_path}.pth")
print(f"Searching for logs file in: {logs_path}")

if weights_files and any(glob_result := glob.glob(logs_path)):
    log_file = glob_result[0]
    output_folder = "/content/toHF"
    os.makedirs(output_folder, exist_ok=True)
    output_zip_path = f"{output_folder}/{experiment_name}.zip"
    with zipfile.ZipFile(output_zip_path, 'w') as zipf:
        for weights_file in weights_files:
            zipf.write(weights_file, os.path.basename(weights_file))
        zipf.write(log_file, os.path.basename(log_file))
    print(f"The files have been added to the zip folder: {output_zip_path}")
else:
    print(f"Found weights files: {weights_files}")
    print(f"Found logs files: {glob_result}")
    raise Exception("Couldn't find your model files. Check the found file results above. (Did you run Index Training?)")

from huggingface_hub import login, HfApi
#@markdown Enter HuggingFace Token (set Role to 'write'):
hftoken = "Obtain this at https://huggingface.co/settings/tokens" #@param {type:"string"}

# Login
login(token=f"{hftoken}")

#@markdown Set this option to true if you wish to create a new repository for this model.
create_new_repo = False #@param {type:"boolean"}
#@markdown Enter the HuggingFace repository name you wish to send the model to:
repoid = "username/RepoName" #@param {type:"string"}

if create_new_repo:
    from huggingface_hub import create_repo
    create_repo(repoid)
    print(f"Repo '{repoid}' created successfully!")

api = HfApi()
# Upload folders
api.upload_folder(folder_path="/content/toHF",
                 repo_id=f"{repoid}",
                 repo_type="model")

# **Inference (EXPERIMENTAL)**

In [ ]:
#@markdown #Step 2. Download The Model
#@markdown Link the URL path to the model (Mega, Drive, etc.) and start the code

from mega import Mega
import os
import shutil
from urllib.parse import urlparse, parse_qs
import urllib.parse
from google.oauth2.service_account import Credentials
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import hashlib

def calculate_md5(file_path):
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()
config_path = '/content/RVC-Disconnected/peppy-generator-388800-07722f17a188.json'

condition1 = False
condition2 = False
already_downloaded = False

# condition1 here is to check if the .index was imported. 2 is for if the .pth was.

!rm -rf /content/unzips/
!rm -rf /content/zips/
!mkdir /content/unzips
!mkdir /content/zips

def sanitize_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            if filename == ".DS_Store" or filename.startswith("._"):
                os.remove(file_path)
        elif os.path.isdir(file_path):
            sanitize_directory(file_path)

url = 'https://huggingface.co/medicreal/MedicAI/resolve/main/EminemMMLP2.zip'  #@param {type:"string"}
model_zip = urlparse(url).path.split('/')[-2] + '.zip'
model_zip_path = '/content/zips/' + model_zip

#private_model = False #@param{type:"boolean"}

if url != '':
    MODEL = ""  # Initialize MODEL variable
    !mkdir -p /content/RVC-Disconnected/logs/$MODEL
    !mkdir -p /content/zips/
    !mkdir -p /content/RVC-Disconnected/weights/  # Create the 'weights' directory

    if "drive.google.com" in url:
        !gdown $url --fuzzy -O "$model_zip_path"
    elif "/blob/" in url:
        url = url.replace("blob", "resolve")
        print("Resolved URL:", url)  # Print the resolved URL
        !wget "$url" -O "$model_zip_path"
    elif "mega.nz" in url:
        m = Mega()
        print("Starting download from MEGA....")
        m.download_url(url, '/content/zips')
    elif "/tree/main" in url:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        temp_url = ''
        for link in soup.find_all('a', href=True):
            if link['href'].endswith('.zip'):
                temp_url = link['href']
                break
        if temp_url:
            url = temp_url
            print("Updated URL:", url)  # Print the updated URL
            url = url.replace("blob", "resolve")
            print("Resolved URL:", url)  # Print the resolved URL

            if "huggingface.co" not in url:
                url = "https://huggingface.co" + url

            !wget "$url" -O "$model_zip_path"
        else:
            print("No .zip file found on the page.")
            # Handle the case when no .zip file is found
    else:
        !wget -q "$url" -O "$model_zip_path"

    for filename in os.listdir("/content/zips"):
        if filename.endswith(".zip"):
            zip_file = os.path.join("/content/zips", filename)
            shutil.unpack_archive(zip_file, "/content/unzips", 'zip')

sanitize_directory("/content/unzips")

def find_pth_file(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".pth"):
                file_name = os.path.splitext(file)[0]
                if file_name.startswith("G_") or file_name.startswith("P_"):
                    config_file = os.path.join(root, "config.json")
                    if os.path.isfile(config_file):
                        print("Outdated .pth detected! This is not compatible with the RVC method. Find the RVC equivalent model!")
                    continue  # Continue searching for a valid file
                file_path = os.path.join(root, file)
                if os.path.getsize(file_path) > 100 * 1024 * 1024:  # Check file size in bytes (100MB)
                    print("Skipping unusable training file:", file)
                    continue  # Continue searching for a valid file
                return file_name
    return None

MODEL = find_pth_file("/content/unzips")
if MODEL is not None:
    print("Found .pth file:", MODEL + ".pth")
else:
    print("Error: Could not find a valid .pth file within the extracted zip.")
    print("If there's an error above this talking about 'Access denied', try one of the Alt URLs in the Google Sheets for this model.")
    MODEL = ""
    global condition3
    condition3 = True

index_path = ""

def find_version_number(index_path):
    if condition2 and not condition1:
        if file_size >= 55180000:
            return 'RVC v2'
        else:
            return 'RVC v1'

    filename = os.path.basename(index_path)

    if filename.endswith("_v2.index"):
        return 'RVC v2'
    elif filename.endswith("_v1.index"):
        return 'RVC v1'
    else:
        if file_size >= 55180000:
            return 'RVC v2'
        else:
            return 'RVC v1'

if MODEL != "":
    # Move model into logs folder
    for root, dirs, files in os.walk('/content/unzips'):
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith(".index"):
                print("Found index file:", file)
                condition1 = True
                logs_folder = os.path.join('/content/RVC-Disconnected/logs', MODEL)
                os.makedirs(logs_folder, exist_ok=True)  # Create the logs folder if it doesn't exist

                # Delete identical .index file if it exists
                if file.endswith(".index"):
                    identical_index_path = os.path.join(logs_folder, file)
                    if os.path.exists(identical_index_path):
                        os.remove(identical_index_path)

                shutil.move(file_path, logs_folder)
                index_path = os.path.join(logs_folder, file)  # Set index_path variable

            elif "G_" not in file and "D_" not in file and file.endswith(".pth"):
                destination_path = f'/content/RVC-Disconnected/weights/{MODEL}.pth'
                if os.path.exists(destination_path):
                    print("You already downloaded this model. Re-importing anyways..")
                    already_downloaded = True
                shutil.move(file_path, destination_path)
                condition2 = True
                if already_downloaded is False and os.path.exists(config_path):
                    file_size = os.path.getsize(destination_path) # Get file size
                    md5_hash = calculate_md5(destination_path) # Calculate md5 hash
                    index_version = find_version_number(index_path)  # Get the index version

if condition1 is False:
    logs_folder = os.path.join('/content/RVC-Disconnected/logs', MODEL)
    os.makedirs(logs_folder, exist_ok=True)
# this is here so it doesnt crash if the model is missing an index for some reason

if condition2 and not condition1:
    print("Model partially imported! No .index file was found in the model download. The author may have forgotten to add the index file.")
    if already_downloaded is False and os.path.exists(config_path) and not private_model:
        update_sheet(url, MODEL, file_size, md5_hash, index_version)

elif condition1 and condition2:
    print("Model successfully imported!")
    if already_downloaded is False and os.path.exists(config_path) and not private_model:
        update_sheet(url, MODEL, file_size, md5_hash, index_version)

elif condition3:
    pass  # Do nothing when condition3 is true
else:
    print("URL cannot be left empty. If you don't want to download a model now, just skip this step.")

!rm -r /content/unzips/
!rm -r /content/zips/

Found .pth file: model.pth
You already downloaded this model. Re-importing anyways..
Found index file: model.index
Model successfully imported!


Upload your audo files to ```/content/Mangio-RVC-Fork/audios```

In [ ]:
#@markdown # Run inference (Using CLI)

!python infer-web.py --pycmd python --is_cli

#@markdown This uses the Gradio CLI, allowing you to run inference using your own models. Please note that this feature is experimental.

#@markdown **Instructions:**

#@markdown **1.** Run the cell.

#@markdown **2.** Type ```go infer```.

#@markdown **3.** Copy the following format:

#@markdown ```
#@markdown modelname.pth audios/input.wav output.wav logs/[modelname]/added_IVF574_Flat_nprobe_1_modelname_v2.index 0 0 rmvpe 160 3 0 0.75 0.7 0.33 0.45 False 8.0 1.2
#@markdown ```

#@markdown Once done, you can download the WAV file from ```/content/Mangio-RVC-Fork/audio-outputs``` in the left file browser.

# TODO, Changelogs, Credits, and Special Thanks
**TODO**
*   Implement "Export Lowest Points", or implement [ROD](https://github.com/grvyscale/RealtimeOvertrainingDetection) on a seperate thread
*   Implement 48k variant of the new RVCv2 pretrains when it is released
*   Implement additions from [Sonphantrung's fork](https://colab.research.google.com/drive/1o4NbL2pCOkc6s5u_vyIReujwCetraTKb) into this notebook. (autosave, `f""` formatting, preprocessing fusion, other things)

**Changelogs**

**v0.24** - Implemented SimplCup's Ov2Super new pretrains for 32k. Select `use_new_pretrains` to use it.

**v0.23** - Implemented an option to use the new Ov2Super pretrained models by SimplCup. These will only work when training a v2 40k model.

**v0.22** - Hopefully fixed an issue regarding people trying to install RVC twice. Also added a new cell to automatically send a model directly to Huggingface by LollenApe.

Older changelogs can be found at https://rentry.co/RVC-Disconnected_changelogs.

**Credits**
*   [MedicDoesStuff](https://www.youtube.com/@medicdoesstuff/) - modified this notebook
*   [Kit Lemonfoot](https://huggingface.co/Kit-Lemonfoot) - writing this notebook
*   [RVC Project](https://github.com/RVC-Project) - Created RVC, obviously
*   [LJ1995](https://huggingface.co/lj1995) - Pretrained RVC models
*   [Mangio261](https://github.com/Mangio621/) - Creating the Mangio RVC fork
*   [Kalomaze](https://github.com/kalomaze) - Original RVC colab + Mangio tweaks
*   [Alexlnkp](https://github.com/alexlnkp) - Created a more up-to-date variant of the Mangio-RVC fork
*   [LollenApe](https://huggingface.co/lollenape) - Created the "Export Finished Model to HuggingFace" cell.
*   [Pony Preservation Project](https://boards.4channel.org/mlp/catalog#s=Pony%20Preservation%20Project) - for their robust TalkNet and So-Vits-SVC notebooks
*   the Colab team - forcing my hand and making me release this notebook early

**Special Thanks**
*   [Fifteen AI](https://15.ai/) - for getting me into voice AI in the first place
*   [Dacoolkid44](https://huggingface.co/dacoolkid44) / [HoloAI44](https://www.youtube.com/@Holo_AI44) and Hijack / [SANSSWEEP](https://huggingface.co/SANSSWEEP) - for basically kickstarting the larger VTuber voice AI scene with their models
*   [Maki Ligon](https://www.youtube.com/@Shiina_Mashiro) / [Yuuto Ichika](https://www.youtube.com/@yuutoichika) - for keeping me grounded in reality while developing this thing
*   [Bartezes](https://www.youtube.com/@bartezes3082) - for helping me so much with the [VTuber AI Model Tracking spreadsheet](https://docs.google.com/spreadsheets/d/1tvZSggOsZGAPjbMrWOAAaoJJFpJuQlwUEQCf5x1ssO8/)
*   [Megaaziib](https://www.youtube.com/@megaaziib) - for inspiring me to keep working on AI models and covers (I don't hate you)
*   [Saintlysaint](https://www.youtube.com/@SaintlySaint) and [Gengar2525](https://www.youtube.com/@GeGaCh) - for having faith in me
